In [42]:
import pandas as pd
import numpy as np
import sqlite3
import datetime
import sklearn
import sqlalchemy as sa
from sklearn.model_selection import TimeSeriesSplit

In [109]:
from collections import Counter
from sklearn.base import BaseEstimator
from tqdm import trange

In [3]:
trans_feat = pd.read_csv('../data/data1per.csv', index_col=0)

/home/alex/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/alex/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
trans_feat.head()

,cdf_s_123,cdf_s_124,cdf_s_127,cdf_s_135,cdf_s_130,cdf_s_129,cdf_s_134,cdf_s_128,cdf_s_138,cdf_s_126,...,user_acct_number_hashed,user_agent_string_hash,browser_plugins_hash,screen_hash,user_id,ip_address,ip_country,ip_region,ip_city,ip_isp
0,NaN,NaN,30,1,30,30,4,NaN,null,null,...,NaN,NaN,NaN,NaN,100003,31.173.83.80,ru,48,Moscow,7255560.0
1,NaN,NaN,30,4,30,30,4,NaN,null,null,...,NaN,NaN,NaN,NaN,100003,185.79.100.99,ru,48,Moscow,6938077.0
2,NaN,NaN,30,5,30,30,30,NaN,null,null,...,NaN,NaN,NaN,NaN,100003,94.25.177.185,ru,48,Moscow,7255560.0
3,NaN,NaN,30,0,30,30,30,NaN,null,null,...,NaN,NaN,NaN,NaN,100003,46.191.68.110,by,03,Grodno,190563.0
4,NaN,NaN,30,30,30,30,0,NaN,null,null,...,NaN,NaN,NaN,NaN,100003,37.110.213.28,uz,13,Tashkent,5860113.0


In [ ]:
data = df_from_sql("""select channel_indicator_desc,
                             cdf_s_138,
                             cdf_s_126,
                             cdf_s_218,
                             event_description,
                             cdf_s_136,
                             cdf_s_137,
                             cdf_s_140,
                             amount_original,
                             custom_mark from rsa_event_log LIMIT 100000""",
                   engine=conn, parse_dates=None)

In [22]:
trans_feat['short_date'] = trans_feat.event_time.apply(lambda x: pd.to_datetime(x).date())

In [25]:
trans_feat['TS_indexer'] = trans_feat.short_date
unique_TS_split = np.array(sorted(trans_feat.TS_indexer.unique()))

In [75]:
def cust_mark_to_class(custom_mark):
    """
    Преобразует входящее значение CUSTOM_MARK в класс
    return:
        1 - фрод
        0 - легитимная
        -1 - неизвестно
    """
    ret = -1
    if custom_mark in ['F','S']:
        ret = 1
    elif custom_mark in ['A','G', np.NaN]:
        ret = 0
    
    return ret

In [81]:
trans_feat['lable'] = trans_feat.custom_mark.apply(lambda x: cust_mark_to_class(x))

In [49]:
len(unique_TS_split)

233

In [95]:
def iterate_megabatches(inputs, n_splits):
    batchsize = len(unique_TS_split)//n_splits
    for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize):
        excerpt = [start_idx, start_idx + batchsize-1]
        yield inputs[excerpt]

In [106]:
batch_df_i_neg = pd.DataFrame()
batch_df_pos = pd.DataFrame()
for train_index in iterate_megabatches(inputs=unique_TS_split, n_splits=3):
    batch_df_i_old_neg = batch_df_i_neg
    batch_df_i = trans_feat[trans_feat.TS_indexer.isin(train_index)]
    batch_df_i_pos = batch_df_i[batch_df_i.lable==1]
    batch_df_i_neg = batch_df_i[batch_df_i.lable==0]
    batch_df_pos = pd.concat([batch_df_pos, batch_df_i_pos])
    batch_df_neg = pd.concat([batch_df_i_old_neg, batch_df_i_neg])
    df_i = pd.concat([batch_df_pos, batch_df_neg])
    #df_balanced_i = 
    #model = xgbclassifier()
    #model.fit(df_balanced_i)
    #pr_i = model.predict()

100%|██████████| 3/3 [00:02<00:00,  1.25it/s]


In [ ]:
Counter(sub_y).most_common(1)[0][0]

In [ ]:
from collections import Counter
from sklearn.base import BaseEstimator

class forget_model(BaseEstimator):
    def __init__(self, feature_types=['categorical'], max_depth=None,
                 min_samples_split=None, min_samples_leaf=None):
        if np.any(list(map(lambda x: x != "real" and x != "categorical",
                           feature_types))):
            raise ValueError("There is unknown feature type")

        self._tree = {}
        self.feature_types = feature_types
        self._max_depth = max_depth
        self._min_samples_split = min_samples_split
        self._min_samples_leaf = min_samples_leaf

    def _fit_node(self, sub_X, sub_y, node):
        if np.all(sub_y == sub_y[0]):
            node["type"] = "terminal"
            node["class"] = sub_y[0]
            return
        
        if len(self.feature_types) == 1:
            ft = np.array(self.feature_types*sub_X.shape[1])
        else:
            ft = self.feature_types
            
        feature_best, threshold_best, gini_best, split = None, None, None, None
        for feature in range(0, sub_X.shape[1]):
            feature_type = ft[feature]
            categories_map = {}

            if feature_type == "real":
                feature_vector = sub_X[:, feature]
            elif feature_type == "categorical":
                counts = Counter(sub_X[:, feature])
                clicks = Counter(sub_X[sub_y == 1, feature])
                ratio = {}
                for key, current_count in counts.items():
                    if key in clicks:
                        current_click = clicks[key]
                    else:
                        current_click = 0
                    ratio[key] = current_click/current_count
                sorted_categories = list(map(lambda x: x[0],
                                             sorted(ratio.items(),
                                                    key=lambda x: x[1])))
                categories_map = dict(zip(sorted_categories,
                                          list(range(len(sorted_categories)))))

                feature_vector = np.array(list(map(lambda x: categories_map[x], sub_X[:, feature])))
            else:
                raise ValueError

            if len(feature_vector) == 0:
                continue

            _, _, threshold, gini = find_best_split(feature_vector, np.array(sub_y))
            if gini_best is None or gini > gini_best:
                feature_best = feature
                gini_best = gini
                split = feature_vector < threshold

                if feature_type == "real":
                    threshold_best = threshold
                elif feature_type == "categorical":
                    threshold_best = list(map(lambda x: x[0],
                                              filter(lambda x: x[1] < threshold,
                                                     categories_map.items())))
                else:
                    raise ValueError

        if gini_best == -1:
            node["type"] = "terminal"
            node["class"] = Counter(sub_y).most_common(1)[0][0]
            return

        if feature_best is None:
            node["type"] = "terminal"
            node["class"] = Counter(sub_y).most_common(1)[0][0]
            return

        node["type"] = "nonterminal"

        node["feature_split"] = feature_best
        if ft[feature_best] == "real":
            node["threshold"] = threshold_best
        elif ft[feature_best] == "categorical":
            node["categories_split"] = threshold_best
        else:
            raise ValueError
        node["left_child"], node["right_child"] = {}, {}
        self._fit_node(sub_X[split], sub_y[split], node["left_child"])
        self._fit_node(sub_X[np.logical_not(split)],
                       sub_y[np.logical_not(split)], node["right_child"])

    def _predict_node(self, x, node):
        
        if len(self.feature_types) == 1:
            ft = np.array(self.feature_types*len(x))
        else:
            ft = self.feature_types
            
        x = np.array(x)
        tree = node
        class_i = None
        for i in range(100):
            if tree['type'] == 'terminal':
                class_i = tree['class']
                break
            else:
                feature_split_i = tree['feature_split']
                if ft[tree['feature_split']] == 'real':
                    real_split_i = tree['threshold']
                    if x[feature_split_i] < real_split_i:
                        tree = tree['left_child']
                    else:
                        tree = tree['right_child']
                else:
                    categories_split_i = tree['categories_split']
                    if x[feature_split_i] in categories_split_i:
                        tree = tree['left_child']
                    else:
                        tree = tree['right_child']
        return class_i

    def fit(self, X, y):
        self._fit_node(X, y, self._tree)

    def predict(self, X):
        predicted = []
        for x in X:
            predicted.append(self._predict_node(x, self._tree))
        return np.array(predicted)

